In [1]:
#TODO add wind bearing relative to rider
#TODO consider #pip install gpxpy or just implement the maths
#TODO create class based structure for the data and functions
#TODO darksky implements minute by minute for one hour and hour by hour for 48 hours
#imports
import numpy as np
import tcxparser
import csv,json,requests,os,time,calendar
from pprint import pprint
from datetime import datetime

In [2]:
class tcxWeather:
    
    
    def __init__(self, xmlfile='demoRoute.tcx'):
        self.raw = tcxparser.TCXParser(xmlfile)
        self.stravaTime = self.raw.time_values()
        self.length = len(self.stravaTime)
        self.latitude = self.raw.latitude_points()
        self.longitude = self.raw.longitude_points()
        self.distance = self.raw.distance_points()
        self.distanceTotal = self.distance[self.length-1]
        self.__bearing__()      
            
            
    def __bearing__(self):
        self.bearing = list()
        self.bearing.append(0) #first bearing 0
        φ = list()
        λ = list()
        for x in self.latitude:
            φ.append(np.deg2rad(x))
        for x in self.longitude:
            λ.append(np.deg2rad(x))
        
        for x in range(1,self.length):
            y = np.sin(λ[x]-λ[x-1]) * np.cos(φ[x])
            x = np.cos(φ[x-1])*np.sin(φ[x]) - np.sin(φ[x-1])*np.cos(φ[x])*np.cos(λ[x]-λ[x-1])
            self.bearing.append(np.degrees(np.arctan2(y, x))%360)
       
    
    def __bearingdec__(self): #fix to one function for both bearings
        self.bear = list()
        self.bear.append(0) #first bearing 0
        φ = list()
        λ = list()
        for x in self.lat:
            φ.append(np.deg2rad(x))
        for x in self.lon:
            λ.append(np.deg2rad(x))
        
        for x in range(1,len(self.lat)):
            y = np.sin(λ[x]-λ[x-1]) * np.cos(φ[x])
            x = np.cos(φ[x-1])*np.sin(φ[x]) - np.sin(φ[x-1])*np.cos(φ[x])*np.cos(λ[x]-λ[x-1])
            self.bear.append(np.degrees(np.arctan2(y, x))%360) #0-360 instead of -180:180
            
       
    def speed(self,**kwargs):
        mps_mph = 2.23694
        mps_kph = 3.6
        if 'mph' in kwargs:
            self.mph = kwargs['mph']
            self.mps = self.mph/mps_mph
            self.kph = self.mps*mps_kph
        elif 'kph' in kwargs:
            self.kph = kwargs['kph']
            self.mps = self.kph/mps_kph
            self.mph = self.mps*mps_mph
        elif 'mps' in kwargs:
            self.mps = kwargs['mps']
            self.kph = self.mps*mps_kph
            self.mph = self.mps*mps_mph
        self.__time__()
        
        
            
            
            
    def decimate(self,**kwargs): #todo add variable input parameters such as dist=10km or time = 1hr or numpoints = 20
        if 'Distance' in kwargs:
            distance = kwargs['Distance']
            #points not constant this is average
            numPoints = self.distanceTotal/distance
            
        if 'Points' in kwargs:
            numPoints = kwargs['Points']
        if 'Time' in kwargs:
            if hasattr(self, 'mps'):
                numPoints = self.totalTime/kwargs['Time']
            else:
                raise Exception('no speed defined use x.Speed(mps =y | kph =z | mph =w)')
            
        else:
            numPoints = 10
        numPoints = np.floor(numPoints).astype(int)
        print('Decimating to {0} Points'.format(numPoints))   
        ind = np.linspace(0, (self.length-1),numPoints, endpoint=True, retstep=False, dtype=None)
        ind = np.floor(ind)
        ind = ind.astype(int)
        self.len = numPoints
        self.lat = np.array(self.latitude)
        self.lon = np.array(self.longitude)
        self.dist = np.array(self.distance)
        self.lat = self.lat[np.ix_(ind)]
        self.lon = self.lon[np.ix_(ind)]
        self.dist = self.dist[np.ix_(ind)]
        self.__bearingdec__()
        self.__timeDec__()
        
    def __timeDec__(self):
        self.timeSecs = list()
        self.timeMin = list()
        self.timeHr = list()
        self.timeSecs.append(0)
        self.timeMin.append(0)
        self.timeHr.append(0)
        time=0
        for x in range(1,self.len):
            delDist = self.dist[x]-self.dist[x-1]
            time += delDist/self.mps
            self.timeMin.append(int (np.round(time/60)))
            self.timeHr.append(int (np.round(time/3600)))
            self.timeSecs.append(int (time))
        
    def __time__(self):
        self.timeSeconds = list()
        self.timeSeconds.append(0)
        time=0
        for x in range(1,self.length):
            delDist = self.distance[x]-self.distance[x-1]
            time += delDist/self.mps
            self.timeSeconds.append(int (time))
        self.totalTime = time           
        
        
    def getWeatherData(self,apikey='none',units='si',writeF=False):
        if hasattr(self,'weatherData'):
             raise Exception('Data already exists')
        self.weatherData = list()
        
        if hasattr(self, 'len'):
            print('Gathering weather data...')
        else:
            raise Exception('Data not decimated not making API call')

        
        for x in range(0,self.len):
            url='https://api.darksky.net/forecast/{0}/{1},{2}?exclude=daily,alerts,flags?units={3}'.format(apikey,self.lat[x],self.lon[x],units)
            data = (requests.get(url).content)
            if writeF:
                if not os.path.exists('weatherData'):
                    os.makedirs('weatherData')
                file = open('weatherData/weatherdataDemoTCX{0}.json'.format(x),'wb')     
                file.write(data)
                file.close
            self.weatherData.append(json.loads(data))
        print('Gathered weather data')
    
    
    def loadExistingData(self,location):
        if hasattr(self,'weatherData'):
             raise Exception('Data already exists')
        self.weatherData = list()
                   
        for x in range(0,self.len):
            filename = '{0}{1}.json'.format(location,x)
            #print('Loading' filename)
            with open(filename) as data_file:  
                self.weatherData.append(json.load(data_file))

    def clearWeatherData(self):
        del self.weatherData
            
    def getForecast(self):
        self.precipIntensity= list()
        for x in range(0,self.len):
            self.precipIntensity.append(self.weatherData[x]["hourly"]["data"][self.timeHr[x]]["precipIntensity"])
            

In [3]:
steve= tcxWeather()
steve.speed(kph=25)
steve.decimate(Time=1000)
steve.loadExistingData('weatherData/weatherdataDemoTCX')
steve.timeHr
steve.getForecast()
steve.precipIntensity
#steve.getWeatherData(apikey='XXXXXXXXXX',units='si',writeF=True)

Decimating to 9 Points


[0, 0, 0.16, 0.1676, 0.1651, 0.0203, 0.0254, 0.0279, 0.0254]

In [5]:
epoch_time = int(time.time())
epoch_time
ride_start_hour = 18
ride_start_minute =40
value = datetime.fromtimestamp(epoch_time)
hour = (value.strftime('%H'))
minute = (value.strftime('%M')) 
second = (value.strftime('%S')) 
hour = int (hour)
minute = int (minute)
second = int (second)
seconds = hour*60*60+minute*60+second
epoch_time = int(time.time())
epoch_time

epoch_start = epoch_time+ride_start_hour*60*60+ride_start_minute*60-seconds
epoch_start

#work out speed 
#work out arrival time at each block 

1489430400

In [ ]:
pprint(steve.weatherData[0]["hourly"]["data"][0]["precipIntensity"])
pprint(steve.weatherData[8]["hourly"]["data"][0]["precipIntensity"])

In [ ]:
steve.speed(mps=5)

In [7]:
np.round(10)

10